In [73]:
%pip install pandas
%pip install openpyxl
%pip install yfiles_jupyter_graphs


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [74]:
import pandas as pd
import os
from yfiles_jupyter_graphs import GraphWidget


In [75]:
EPOCH_ROW = 0
CYCLE_ROW = 1
CYCLE_START_ROW = 2
CYCLE_PERIOD_ROW = 3
CYCLE_END_RULE_ROW = 4
TIMING_ROW = 5
VISIT_LABEL_ROW = 6
VISIT_WINDOW_ROW = 7

FIRST_ACTIVITY_ROW = 8

FIRST_VISIT_COL = 4

In [76]:
def get_cycle_cell(df, row_index, col_index):
  is_null = pd.isnull(df.iloc[row_index, col_index])
  if is_null:
    return "", True
  else:
    value = str(df.iloc[row_index, col_index])
    if value.upper() == "-":
      return "", True
    else:
      return value, False

def previous_index(index):
  if index == 0:
    return 0
  else:
    return index - 1

def build_cycle_record(df, index, col_index, condition):
  cycle_start_index = index
  cycle_start, is_null = get_cycle_cell(df, CYCLE_START_ROW, col_index)
  cycle_period, is_null = get_cycle_cell(df, CYCLE_PERIOD_ROW, col_index)
  cycle_end_rule, is_null = get_cycle_cell(df, CYCLE_END_RULE_ROW, col_index)
  return { 
    'start_index': cycle_start_index, 
    'condition': condition, 
    'start': cycle_start, 
    'period': cycle_period, 
    'end_rule': cycle_end_rule 
  }


In [77]:
def extract_cycles(df):
  cycles = []
  timepoint_index = -1
  cycle_start_index = None
  in_cycle = False
  prev_cycle = None
  for col_index in range(df.shape[1]):
    if col_index >= FIRST_VISIT_COL:
      timepoint_index += 1
      cycle, cycle_is_null = get_cycle_cell(df, CYCLE_ROW, col_index)
      if cycle_is_null:
        if in_cycle:
          cycle_record['end_index'] = previous_index(timepoint_index)
          cycles.append(cycle_record)
          in_cycle = False
        else:
          pass # Do nothing
      else:
        cycle = str(cycle)
        if not in_cycle:
          in_cycle = True
          cycle_record = build_cycle_record(df, timepoint_index, col_index, cycle)
        elif prev_cycle == cycle:
          pass # Do nothing
        else:
          cycle_record['end_index'] = previous_index(timepoint_index)
          cycles.append(cycle_record)
          cycle_record = build_cycle_record(df, timepoint_index, col_index, cycle)
      prev_cycle = cycle
  return cycles

def extract_timepoints(df):
  timepoints = []
  timepoint_index = -1
  for col_index in range(df.shape[1]):
    if col_index >= FIRST_VISIT_COL:
      timepoint_index += 1
      timing, timing_is_null = get_cycle_cell(df, TIMING_ROW, col_index)
      window, window_is_null = get_cycle_cell(df, VISIT_WINDOW_ROW, col_index)
      timepoints.append({ 'value': timing, 'type': "A", 'window': window})
  return timepoints


In [78]:
def custom_node_color(index: int, node: dict):
  if 'node_type' in node['properties']:
    if node['properties']['node_type'] == 'entry_exit':
      return 'black'
    elif node['properties']['node_type'] == 'anchor':
      return 'black'
    elif node['properties']['node_type'] == 'condition':
      return 'black'
    elif node['properties']['node_type'] == 'timepoint':
      return 'black'
    elif node['properties']['node_type'] == 'visit':
      return '#c1141a'
    elif node['properties']['node_type'] == 'activity':
      return '#1555bd'
    elif node['properties']['node_type'] == 'bc':
      return '#c0d6e4'
    else:
      return 'white'
  else: 
    return 'white'

def custom_node_style(index: int, node: dict):
  if 'node_type' in node['properties']:
    if node['properties']['node_type'] == 'entry_exit':
      #return { 'shape': 'round-rectangle' }
      return {
        'image': 'https://raw.githubusercontent.com/data4knowledge/timepoints/main/images/pill_black.svg'
        #'image': 'https://gist.githubusercontent.com/fskpf/b5c5b765139056ddc7e72ea28d4f44e4/raw/f4483469a9d4f638a8acae39aa6adfd76b61f587/yfiles-jupyter-graphs-icon.svg'
      }
    elif node['properties']['node_type'] == 'anchor':
      return { 'shape': 'hexagon2' }
    elif node['properties']['node_type'] == 'condition':
      return { 'shape': 'diamond' }
    elif node['properties']['node_type'] == 'timepoint':
      return { 'shape': 'ellipse' }
    elif node['properties']['node_type'] == 'visit':
      return { 'shape': 'ellipse' }
    elif node['properties']['node_type'] == 'activity':
      return { 'shape': 'ellipse' }
    elif node['properties']['node_type'] == 'bc':
      return { 'shape': 'ellipse' }
    else:
      return { 'shape': 'ellipse' }
  else: 
    return { 'shape': 'ellipse' }


In [79]:
notebook_path = os.path.abspath("notebook.ipynb")
file_path = os.path.join(os.path.dirname(notebook_path), "source_data/berber_1_v2.xlsx")
#file_path = os.path.join(os.path.dirname(notebook_path), "source_data/Roche Phase 3 NCT04320615.xlsx")
df = pd.read_excel(file_path, header=None)
df = df.fillna(method='ffill', axis=1)
cycles = extract_cycles(df)
print(cycles)
#nodes, edges = prepare_nodes_and_edges(raw_data)

[{'start_index': 1, 'condition': '1', 'start': 'Day 1', 'period': '15 Days', 'end_rule': '', 'end_index': 2}, {'start_index': 3, 'condition': '2', 'start': 'Day 16', 'period': '15 Days', 'end_rule': '', 'end_index': 4}, {'start_index': 5, 'condition': '3', 'start': 'Day 31', 'period': '15 Days', 'end_rule': '', 'end_index': 6}, {'start_index': 7, 'condition': '4..12', 'start': 'Day 46', 'period': '15 Days', 'end_rule': '', 'end_index': 7}, {'start_index': 8, 'condition': '13 .. N step 3', 'start': 'Day 211', 'period': '15 Days', 'end_rule': 'Disease Progresssion', 'end_index': 8}]


In [80]:
#w = GraphWidget()
#w.orthogonal_layout()
#w.set_directed(True)

#w.set_nodes(nodes)
#w.set_edges(edges)

#w.set_node_color_mapping(custom_node_color)
#w.set_node_styles_mapping(custom_node_style)
#w